# imports

In [1]:
import pysubgroup as ps
import pandas as pd
import numpy as np
import import_ipynb
import os
import csv
from pmdarima import auto_arima

# function defs

In [2]:
def resample_array(arr, fixed_length):
    """
    Resample a 1D numpy array to a fixed length using linear interpolation.

    Parameters:
        arr (array-like): Original array of numeric values.
        fixed_length (int): Desired length of the resampled array.

    Returns:
        numpy.ndarray: Resampled array of length 'fixed_length'.
    """
    arr = np.asarray(arr, dtype=float)
    original_length = len(arr)

    if original_length == fixed_length:
        return arr

    # Create original and new equally spaced indices (scaled from 0 to 1)
    original_indices = np.linspace(0, 1, original_length)
    target_indices = np.linspace(0, 1, fixed_length)

    # Use linear interpolation to compute resampled values
    resampled = np.interp(target_indices, original_indices, arr)

    return resampled

In [ ]:
def run_sd_array_magnitude(df, initial_data, target_col, fixed_length=None, min_size_sg = None, alpha = None, result_set_size=None, depth=None):
    if result_set_size is None:
        result_set_size = 1
    if depth is None:
        depth = 10
    if fixed_length is None:
        fixed_length = 8
    if min_size_sg is None:
        min_size_sg = 100
    if alpha is None:
        alpha = 0.5

    data = df
    target = ps.ArrayTarget(target_col, fixed_length = fixed_length, initial_data= initial_data)
    searchspace = ps.create_selectors(data, ignore=target_col)
    task = ps.SubgroupDiscoveryTask (
        data,
        target,
        searchspace,
        result_set_size=result_set_size,
        depth=depth,
        qf=ps.ArraySignMagnitudeQF(alpha = alpha, fixed_length = fixed_length, min_size_sg = min_size_sg, initial_data= initial_data))
    result = ps.DFS().execute(task)

    return result.to_dataframe()

In [4]:
def parse_array_from_string(s):
    """
    Parse a string representing an array of numbers separated by whitespace or commas.
    
    Examples:
        s = "[-1.33 -1.63 -1.94 -2.37]"
        s = "[-0.82, 0.58, 2.72, 1.89]"
    
    Returns:
        np.array: A NumPy array of floats.
    """
    # Remove surrounding whitespace and brackets
    s = s.strip()
    if s.startswith("[") and s.endswith("]"):
        s = s[1:-1].strip()
    elif s.startswith('"[') and s.endswith(']"'):
        s = s[2:-2].strip()

    # Replace commas with spaces so we can split regardless of separator
    s = s.replace(",", " ")

    try:
        numbers = [float(item) for item in s.split()]
    except ValueError as e:
        raise ValueError(f"Error parsing string into floats: {s}") from e

    return np.array(numbers)

In [5]:
def get_attr_array(init_results):
    quality = init_results.iloc[0, 0]
    sg_des = init_results.iloc[0, 1]
    size_sg = init_results.iloc[0, 2]
    size_cover_all = init_results.iloc[0, 3]
    covered_not_in_sg = init_results.iloc[0, 4]
    size_dataset = init_results.iloc[0, 5]
    centroid_sg = init_results.iloc[0, 6]
    centroid_dataset = init_results.iloc[0, 7]
    avg_cosine = init_results.iloc[0, 8]
    sign_consistency = init_results.iloc[0, 9]

    return quality, sg_des, size_sg,size_cover_all,covered_not_in_sg, size_dataset, centroid_sg, centroid_dataset, avg_cosine, sign_consistency

def append_and_cut_array(best_results, init_results, df, sg_des, size_sg, discovered_so_far):
    best_results.append(init_results.iloc[0])
    mask = sg_des.covers(df)
    remaining_data = df[~mask]
    discovered_so_far += size_sg

    return best_results, remaining_data, discovered_so_far

In [6]:
def peeling_magnitude(df, target_col, alpha_val, min_size_sg_val):
    
    print("Starting new run.")
    best_results = []
    discovered_so_far = 0
    iteration = 1
    new_min = min_size_sg_val
    remaining_data = df
    # Initial subgroup discovery using the magnitude-based quality function.
    init_results = run_sd_array_magnitude(df,df, target_col, alpha=alpha_val, min_size_sg=min_size_sg_val)
    quality, sg_des, size_sg,size_cover_all,covered_not_in_sg, size_dataset, centroid_sg, centroid_dataset, avg_cosine, sign_consistency = get_attr_array(init_results)
    
    if quality < 0.5:
        print('No satisfactory subgroup found, quality = ', quality, " and sg size = ", size_sg, " and remaining data = ", len(remaining_data))
    else:
        print('Initial satisfactory subgroup found, quality = ', quality, " and sg size = ", size_sg, " and remaining data = ", len(remaining_data))
        best_results, remaining_data, discovered_so_far = append_and_cut_array(
            best_results, init_results, df, sg_des, size_sg, discovered_so_far
        )
    
    print("Starting recursion")
    while quality > 0.5 and len(remaining_data) > 100:
        if quality < 0.6:
            new_min = max(5, new_min - 10)
        iteration += 1
        print("Starting iteration: ", iteration)
        result = run_sd_array_magnitude(remaining_data,df, target_col, alpha=alpha_val, min_size_sg=new_min)
        quality, sg_des, size_sg,size_cover_all,covered_not_in_sg, size_dataset, centroid_sg, centroid_dataset, avg_cosine, sign_consistency = get_attr_array(result)
        print("Finished iteration: ", iteration, " with quality = ", quality, " and sg size = ", size_sg, " and remaining data = ", len(remaining_data))
        new_min = min_size_sg_val
        if quality < 0.5 or len(remaining_data) < 100:
            print("Quality fell below 0.5 or remaining data is less than 100, stopping search")
            break
        best_results, remaining_data, discovered_so_far = append_and_cut_array(
            best_results, result, remaining_data, sg_des, size_sg, discovered_so_far
        )
        
    
    print("Ended recursive search, results are final")
    print("Items discovered so far:", discovered_so_far)
    print("Items left to classify:", len(remaining_data))
    return best_results

In [7]:
def greedy_peeling_EMM(df, target_col, alpha_val, min_size_sg_val):
    
    print("Starting new run.")
    best_results = []
    discovered_so_far = 0
    iteration = 1
    new_min = min_size_sg_val
    remaining_data = df
    # Initial subgroup discovery using the magnitude-based quality function.
    init_results = run_EMM(df,df, target_col, alpha=alpha_val, min_size_sg=min_size_sg_val)
    quality, sg_des, size_sg = get_attr_array(init_results)
    
    if quality < 0.5:
        print('No satisfactory subgroup found, quality = ', quality, " and sg size = ", size_sg, " and remaining data = ", len(remaining_data))
    else:
        print('Initial satisfactory subgroup found, quality = ', quality, " and sg size = ", size_sg, " and remaining data = ", len(remaining_data))
        best_results, remaining_data, discovered_so_far = append_and_cut_array(
            best_results, init_results, df, sg_des, size_sg, discovered_so_far
        )
    
    print("Starting recursion")
    while quality > 0.5 and len(remaining_data) > 100:
        if quality < 0.6:
            new_min = max(5, new_min - 10)
        iteration += 1
        print("Starting iteration: ", iteration)
        result = run_EMM(remaining_data,df, target_col, alpha=alpha_val, min_size_sg=new_min)
        quality, sg_des, size_sg = get_attr_array(result)
        print("Finished iteration: ", iteration, " with quality = ", quality, " and sg size = ", size_sg, " and remaining data = ", len(remaining_data))
        new_min = min_size_sg_val
        if quality < 0.5 or len(remaining_data) < 100:
            print("Quality fell below 0.5 or remaining data is less than 100, stopping search")
            break
        best_results, remaining_data, discovered_so_far = append_and_cut_array(
            best_results, result, remaining_data, sg_des, size_sg, discovered_so_far
        )
        
    
    print("Ended recursive search, results are final")
    print("Items discovered so far:", discovered_so_far)
    print("Items left to classify:", len(remaining_data))
    return best_results

In [8]:
def get_sg_dfs(wide_df, results_df):
    # Dictionary to hold one DataFrame per subgroup
    subgroup_dfs = {}
    chopped_data = wide_df
    # Loop through each row in cosine_array_recursive_results_df.
    # It is assumed that each row has a subgroup description in the "subgroup" column.
    for idx, row in results_df.iterrows():
        subgroup_condition = row['subgroup']
        try:
            # Apply the filter to the original data to extract only the rows satisfying the subgroup condition.
            mask = subgroup_condition.covers(chopped_data)
            filtered_df = chopped_data[mask]
            subgroup_dfs[subgroup_condition] = filtered_df
            chopped_data = chopped_data[~chopped_data['ItemNumber'].isin(filtered_df['ItemNumber'])]
            print(f"Subgroup '{subgroup_condition}' has {len(filtered_df)} items.")
        except Exception as e:
            print(f"Error for subgroup '{subgroup_condition}': {e}")
    return subgroup_dfs

In [9]:
def get_sg_dfs_indexed(wide_df, results_df):
    # Dictionary to hold one DataFrame per subgroup
    subgroup_dfs = {}
    chopped_data = wide_df
    # Loop through each row in cosine_array_recursive_results_df.
    # It is assumed that each row has a subgroup description in the "subgroup" column.
    for idx, row in results_df.iterrows():
        subgroup_condition = row['subgroup']
        sg_index = row['Index']
        try:
            # Apply the filter to the original data to extract only the rows satisfying the subgroup condition.
            mask = subgroup_condition.covers(chopped_data)
            filtered_df = chopped_data[mask]
            subgroup_dfs[sg_index] = filtered_df
            chopped_data = chopped_data[~chopped_data['ItemNumber'].isin(filtered_df['ItemNumber'])]
            print(f"Subgroup '{subgroup_condition}' has {len(filtered_df)} items.")
        except Exception as e:
            print(f"Error for subgroup '{subgroup_condition}': {e}")
    return subgroup_dfs

In [10]:
def compute_avg_turnover_per_year(order_data_eu_sg):
    """
    Computes average turnover per year for a given order data subgroup,
    excluding rows with abnormally high prices.

    Parameters:
        order_data_eu_sg (pd.DataFrame): Filtered order data for a subgroup.

    Returns:
        np.ndarray: Array of average yearly turnover values.
    """

    # Identify high price rows
    high_price_rows = order_data_eu_sg[order_data_eu_sg['PriceSalesUoMEUR'] > 100000]
    if not high_price_rows.empty:
        print("⚠️ Warning: Data error in the group – rows with PriceSalesUoMEUR > 100000 were excluded.")

    # Exclude high price rows
    clean_data = order_data_eu_sg[order_data_eu_sg['PriceSalesUoMEUR'] <= 100000]

    # Compute total turnover per product per year
    turnover_sum_per_product_year = (
        clean_data.groupby(['ItemNumber', 'OrderYear'])['Turnover']
        .sum()
        .reset_index()
    )

    # Compute total turnover per year per semester
    turnover_sum_per_product_year_semester = (
        clean_data.groupby(['ItemNumber', 'OrderYear', 'Semester'])['Turnover']
        .sum()
        .reset_index()
    )

    # Compute average turnover per year
    average_turnover_per_year = (
        turnover_sum_per_product_year.groupby('OrderYear')['Turnover']
        .mean()
        .round()
        .reset_index()
    )

    # Compute average turnover per year per semester
    average_turnover_per_year_semester = (
        turnover_sum_per_product_year_semester.groupby(['OrderYear', 'Semester'])['Turnover']
        .mean()
        .round()
        .reset_index()
    )

    return average_turnover_per_year_semester['Turnover'].to_numpy()


In [11]:
def compute_timespans(order_data_eu_sg):
    # Identify high price rows
    high_price_rows = order_data_eu_sg[order_data_eu_sg['PriceSalesUoMEUR'] > 100000]
    if not high_price_rows.empty:
        print("⚠️ Warning: Data error in the group – rows with PriceSalesUoMEUR > 100000 were excluded.")

    # Exclude high price rows
    clean_data = order_data_eu_sg[order_data_eu_sg['PriceSalesUoMEUR'] <= 100000]

    #exclude other years
    clean_data = clean_data[clean_data['OrderYear'] >= 2021]
    clean_data = clean_data[clean_data['OrderYear'] <= 2024]

    # Compute the time span for each item
     
        # Group by ItemNumber and calculate the earliest and latest order years
    order_year_range = clean_data.groupby('ItemNumber')['OrderYear'].agg(['min', 'max']).reset_index()

    # Rename the columns for clarity
    order_year_range.columns = ['ItemNumber', 'EarliestOrderYear', 'LatestOrderYear']
    order_year_range['Timespan'] = order_year_range['LatestOrderYear'] - order_year_range['EarliestOrderYear'] +1
    mean_timespan = order_year_range['Timespan'].mean()

    timespan_counts = order_year_range['Timespan'].value_counts()

    
    items_introduced_2021_count = timespan_counts.get(4, 0)
    items_introduced_2022_count = timespan_counts.get(3, 0)
    items_introduced_2023_count = timespan_counts.get(2, 0)
    items_introduced_2024_count = timespan_counts.get(1, 0)


    return mean_timespan, items_introduced_2021_count, items_introduced_2022_count, items_introduced_2023_count, items_introduced_2024_count

In [ ]:
def hierarchical_mint_forecast(
    df_wide: pd.DataFrame,
    h: int = 1,
    seasonal: bool = False,
    m: int = 1
) -> pd.DataFrame:
    """
    Perform Hierarchical + (OLS‐style MinT) reconciliation on a subgroup.
    
    Parameters
    ----------
    df_wide : DataFrame
        Historical elasticity panel of shape (T × N), where
        - index = TimePeriod (1…8 or actual dates)
        - columns = ItemNumber
        - values = IndividualPE
    h : int
        Forecast horizon (e.g. 1 for one‐step‐ahead).
    seasonal : bool
        Whether to include seasonal terms in auto_arima.
    m : int
        Seasonal period (only if seasonal=True).
        
    Returns
    -------
    reconciled_df : DataFrame
        Reconciled bottom‐level forecasts of shape (h × N), with the same
        columns as df_wide, and a future index (TimePeriod T+1 … T+h).
    """
    # 1) Build the top‐level (aggregate) series
    #    You can sum or average; we’ll use the mean here:
    agg_series = df_wide.mean(axis=1)

    # 2) Fit ARIMA to the aggregate
    print("Starting ARIMA - training model on subgroup ...")
    model_agg = auto_arima(
        agg_series,
        seasonal=seasonal,
        m=m,
        error_action='ignore',
        suppress_warnings=True
    )
    print("Model trained on subgroup - staring general prediction...")
    #    Produce h‐step‐ahead aggregate forecast
    f_agg = model_agg.predict(n_periods=h)            # shape (h,)
    
    # 3) Fit bottom‐level models (one per product)
    product_ids = df_wide.columns.tolist()
    print(f"General prediction done! - starting individual product training on {len(product_ids)} products")
    bottom_forecasts = []
    count = 0
    passed_75 = False
    passed_50 = False
    passed_25 = False
    for pid in product_ids:
        count+=1
        if count / len(product_ids) >= 0.75 and passed_75 == False:
            print("75% of products trained!")
            passed_75 = True
        if count / len(product_ids) >= 0.5 and passed_50 == False:
            print("50% of products trained!")
            passed_50 = True
        if count / len(product_ids) >= 0.25 and passed_25 == False:
            print("25% of products trained!")
            passed_25 = True

        m_b = auto_arima(
            df_wide[pid],
            seasonal=seasonal,
            m=m,
            error_action='ignore',
            suppress_warnings=True
        )
        bottom_forecasts.append(m_b.predict(n_periods=h))
    
    print("Individual product training done! - starting reconciliation...")

    # Stack into array of shape (N_products × h)
    f_b = np.vstack(bottom_forecasts)                 # shape (N, h)
    n_products = f_b.shape[0]

    # 4) OLS‐style MinT reconciliation (identity W)
    #    Compute per‐horizon correction so bottoms sum to aggregate
    correction = (f_agg - f_b.sum(axis=0)) / n_products  # shape (h,)
    # force it into a 1-D NumPy array
    correction = np.asarray(correction)
    f_b_rec = f_b + correction[np.newaxis, :]           # shape (N, h)

    # 5) Wrap into a DataFrame with a future TimePeriod index
    last_period = df_wide.index.max()
    future_index = range(last_period + 1, last_period + 1 + h)
    reconciled_df = pd.DataFrame(
        f_b_rec.T,               # transpose to (h, N)
        index=future_index,
        columns=product_ids
    )
    print("Reconciliation done! Results are ready.")

    return reconciled_df

In [ ]:
# 1) Define Directional Magnitude Score (DMS)
def directional_magnitude_score(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    train: np.ndarray,
    w: float = 0.5,
    R: float = 25.0
) -> float:
    """
    Directional Magnitude Score (0–1):
      D = 1 if sign(y_pred) == sign(y_true), else 0
      M = max(0, 1 - abs(y_pred - y_true) / R)
      DMS = w * D + (1 - w) * M

    Returns np.nan if train is constant.
    """
    # Exclude constant-series
    if np.allclose(train, train[0]):
        return np.nan

    # Flatten in case of multi-step h > 1
    y_true = np.ravel(y_true)
    y_pred = np.ravel(y_pred)

    scores = []
    for yt, yp in zip(y_true, y_pred):
        # 1) Direction correctness
        D = 1.0 if np.sign(yt) == np.sign(yp) else 0.0
        # 2) Magnitude closeness
        M = max(0.0, 1.0 - abs(yp - yt) / R)
        # 3) Combined score
        scores.append(w * D + (1 - w) * M)

    return float(np.mean(scores))

def evaluate_hierarchical_dms_all(
    subgroup_indices,
    wide_name_tpl,
    folds: list = [5, 6, 7],
    h: int = 1,
    w: float = 0.5,
    R: float = 25.0
):
    """
    For each subgroup:
      • Computes the mean DMS (Directional Magnitude Score) across all products & folds,
        prints it.
      • Stores the mean DMS per product for later analysis.

    Returns
    -------
    subgroup_scores : dict
        { idx -> mean DMS over products }
    per_product_scores : dict
        { idx -> { product_id -> mean DMS for that product } }
    """
    subgroup_scores = {}
    per_product_scores = {}

    for idx in subgroup_indices:
        # grab the wide panel from globals
        df_wide = globals()[wide_name_tpl.format(idx=idx)]
        
        # prepare a place to collect per-fold DMS per product
        errors = {pid: [] for pid in df_wide.columns}

        # rolling‐origin backtest
        for t_end in folds:
            train    = df_wide.iloc[:t_end]
            test     = df_wide.iloc[t_end : t_end + h]
            forecast = hierarchical_mint_forecast(train, h=h)

            for pid in df_wide.columns:
                y_true       = test[pid].values
                y_pred       = forecast[pid].values
                train_series = train[pid].values
                score = directional_magnitude_score(
                    y_true, y_pred, train_series, w=w, R=R
                )
                errors[pid].append(score)

        # average over folds for each product
        mean_per_product = {
            pid: float(np.nanmean(scores))
            for pid, scores in errors.items()
        }
        per_product_scores[idx] = mean_per_product

        # average those product‐means into one subgroup score
        subgroup_mean = float(np.nanmean(list(mean_per_product.values())))
        subgroup_scores[idx] = subgroup_mean

        print(f"Subgroup {idx}: mean DMS = {subgroup_mean:.3f}")

    return subgroup_scores, per_product_scores




In [15]:
def calculate_max_increase(glp_array):
    if isinstance(glp_array, (list, np.ndarray)) and len(glp_array) > 1:
        return max((glp_array[i + 1] - glp_array[i]) / glp_array[i] for i in range(len(glp_array) - 1))
    return np.nan

In [16]:
def rec_for_row(row, i0, i1, k):
    E     = row['Predicted PE']
    p0    = row['LastGLP']
    q0    = row['LastQty']
    f_max = row['MaxIncrease']
    r = i1 / i0
    switch_point = 1.0 / (1.0 - 2.0 * r**2)
    bump = k * E

    # 1) E < switch_point
    if E < switch_point:
        P1_candidate = ((E - 1) / (2*E)) * (i0 / i1) * p0
        Q1_candidate = (1 - E) * q0 / 2

        if P1_candidate * Q1_candidate > p0 * q0:
            return P1_candidate
        else:
            # fallback to indexation
            return p0 * (i1 / i0)

    # 2) switch_point <= E < 0
    if E >= switch_point and E < 0:
        return (i1 / i0) * p0

    # 3) E >= 0
    if f_max >= 0:
        bump = min(bump, f_max)
    return p0 * r * (1 + bump)

In [17]:
def est_qty_per_row(row, i0, i1):
    E     = row['Predicted PE']
    p0    = row['LastGLP']
    p1   = row['Recommended GLP']
    r = i1 / i0
    switch_point = 1.0 / (1.0 - 2.0 * r**2)
    q0   = row['LastQty']

    # 1) E < switch_point
    if E < switch_point and p1<p0:
        q1 = (1-E)*q0/2
    
    elif E < switch_point and p1>=p0:
        q1 =  q0 
    # 2) switch_point <= E < 0
    elif E >= switch_point and E < 0:
        q1 =  q0

    # 3) E >= 0
    else:
        q1 = q0 * ((p1 / p0) * (i1 / i0))**E

    return min(q1, 3*q0)  # cap at 3 times the original quantity

In [18]:
def profit_check_row(row):
    q0 = row['LastQty']
    p0 = row['LastGLP']
    p1 = row['Recommended GLP']
    q1 = row['EstQty']
    return p1*q1 - p0*q0